In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import json
import os


train_data = os.path.join("/content/drive/MyDrive/cifar-10/train.json")
df = pd.read_json(train_data)
train_label = df["Label"].tolist()
train_image = df["Image"].tolist()


test_data = os.path.join("/content/drive/MyDrive/cifar-10/test.json")
df = pd.read_json(test_data)
test_image = df["Image"].tolist()


transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])


# 定義標籤映射
label_mapping = {0: 0, 1: 1, 3: 2, 5: 3, 8: 4}

# 使用空的 train_y 列表，並逐一映射標籤
train_y = []
for label in train_label:
    mapped_label = label_mapping[label]  # 將標籤映射為新的值
    train_y.append(mapped_label)         # 將映射後的標籤加入 train_y 列表


train_x = []

for dataX in train_image:
  dataXs = np.array(dataX, dtype = np.uint8)
  dataXs = Image.fromarray(dataXs)
  train_x.append(transform(dataXs))


#print(train_x)

test_x = []

for data in test_image:
  datas = np.array(data, dtype = np.uint8)
  datas = Image.fromarray(datas)
  test_x.append(transform(datas))

num_workers = 0
#batch size
batch_size = 20

#validation size
n_valid = 0.2

state = np.random.get_state()
np.random.shuffle(train_x)
np.random.set_state(state)
np.random.shuffle(train_y)

train_size = len(train_x)
split = int(np.floor(train_size * n_valid))

train_x, train_y = train_x[split:], train_y[split:]
valid_x, valid_y = train_x[:split], train_y[:split]

classes = ["airplane", "car", "cat", "dog", "ship"]

print(train_x)
print(type(train_x))

In [ ]:
import torch.nn as nn



class EarlyStopping:
    def __init__(self, patience=2, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):

        if self.best_loss is None:
            self.best_loss = val_loss

        elif val_loss > self.best_loss - self.delta:
            self.counter += 1

            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


early_stopping = EarlyStopping(patience=5)

class NN_network(nn.Module):
  def __init__(self):
    super(NN_network, self).__init__()
    #input 3*32*32

    # Convolutional layers

                #nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
    self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
    # => 16*32*32 => pool => 16*16*16

    self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
    # => 32*16*16 => pool => 32*8*8


    # Pooling layer
    self.pool = nn.MaxPool2d(2, 2)

    # Fully Connected Layers
    # Linear layer (32*8*8 -> 512)
    self.fc1 = nn.Linear(32*8*8, 512)

    # Linear layer (512 -> 256)
    self.fc2 = nn.Linear(512, 256)

    # Linear layer (256 -> 5)
    self.fc3 = nn.Linear(256, 5)

    # Dropout
    self.dropout = nn.Dropout(0.25)

    # Softmax
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.conv1(x)
    x = torch.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = torch.relu(x)
    x = self.pool(x)

    x = x.view(-1, 32*8*8)

    x = self.fc1(x)
    x = torch.relu(x)
    x = self.dropout(x)

    x = self.fc2(x)
    x = torch.relu(x)
    x = self.dropout(x)

    x = self.fc3(x)

    x = self.softmax(x)
    return x



# 建立模型
model = NN_network()
print(model)

# 創建優化器
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

# 建立評分標準
criterion = nn.CrossEntropyLoss()

# 設定最大世代數
max_epoch = 20


# 調整 model 至 train 模式
model.train()

train_loss = []
valid_loss = []

for epoch in range(max_epoch):

  train_loss_index = 0.0
  valid_loss_index = 0.0
  for(input, label) in zip(train_x, train_y):

    label = torch.tensor(label, dtype = torch.long).unsqueeze(0)
    input = torch.unsqueeze(input, 0)

    optimizer.zero_grad()      # 將梯度歸零

    y_pred = model(input)

    loss = criterion(y_pred, label)

    loss.backward()         # 透過 backward 取得每個參數的梯度值
    optimizer.step()         # 透過梯度下降執行下一步的參數更新

    train_loss_index += loss.item()*input.size(0)

  model.eval()
  for (input, label) in zip(valid_x, valid_y):
    label = torch.tensor(label, dtype = torch.long).unsqueeze(0)
    input = torch.unsqueeze(input, 0)

    y_pred = model(input)

    loss = criterion(y_pred, label)

    valid_loss_index += loss.item()*input.size(0)

  train_loss_index = train_loss_index/len(train_x)
  valid_loss_index = valid_loss_index/len(valid_x)

  train_loss.append(train_loss_index)
  valid_loss.append(valid_loss_index)

  if not (epoch + 1) % 1:
    print(f'Epoch: {epoch+1} \tTraining Loss: {train_loss_index} \tValidation Loss: {valid_loss_index}')
    #print(f'epoch: {epoch+1}, loss: {loss.item()}')

  early_stopping(valid_loss_index)
  if early_stopping.early_stop:
      print("Early stopping triggered.")
      break
print(model)




NN_network(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=5, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)
Epoch: 1 	Training Loss: 1.1838756848865597 	Validation Loss: 0.8635313077243045
Epoch: 2 	Training Loss: 0.8413447286952049 	Validation Loss: 0.665833334524541
Epoch: 3 	Training Loss: 0.6595656018381674 	Validation Loss: 0.6259807572733361
Epoch: 4 	Training Loss: 0.5152331858654929 	Validation Loss: 0.5926325302633351
Epoch: 5 	Training Loss: 0.4206233589992961 	Validation Loss: 0.42469300451327774
Epoch: 6 	Training Loss: 0.32563536805203636 	Validation Loss: 0.3949077255927687
Epoch: 7 	Training Loss: 0.27282

In [ ]:
import torch.nn as nn



class EarlyStopping:
    def __init__(self, patience=2, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):

        if self.best_loss is None:
            self.best_loss = val_loss

        elif val_loss > self.best_loss - self.delta:
            self.counter += 1

            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


early_stopping = EarlyStopping(patience=5)

class NN_network(nn.Module):
  def __init__(self):
    super(NN_network, self).__init__()
    #input 3*32*32

    # Convolutional layers

                #nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
    self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
    # => 16*32*32 => pool => 16*16*16

    self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
    # => 32*16*16 => pool => 32*8*8


    # Pooling layer
    self.pool = nn.MaxPool2d(2, 2)

    # Fully Connected Layers
    # Linear layer (32*8*8 -> 512)
    self.fc1 = nn.Linear(32*8*8, 512)

    # Linear layer (512 -> 256)
    self.fc2 = nn.Linear(512, 256)

    # Linear layer (256 -> 5)
    self.fc3 = nn.Linear(256, 5)

    # Dropout
    self.dropout = nn.Dropout(0.25)

    # Softmax
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.conv1(x)
    x = torch.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = torch.relu(x)
    x = self.pool(x)

    x = x.view(-1, 32*8*8)

    x = self.fc1(x)
    x = torch.relu(x)
    x = self.dropout(x)

    x = self.fc2(x)
    x = torch.relu(x)
    x = self.dropout(x)

    x = self.fc3(x)

    x = self.softmax(x)
    return x



# 建立模型
model = NN_network()
print(model)

# 創建優化器
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

# 建立評分標準
criterion = nn.CrossEntropyLoss()

# 設定最大世代數
max_epoch = 20


# 調整 model 至 train 模式
model.train()

train_loss = []
valid_loss = []
train_accuracy = []
val_accuracy = []
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))

for epoch in range(max_epoch):

  train_loss_index = 0.0
  valid_loss_index = 0.0
  for(input, label) in zip(train_x, train_y):

    label = torch.tensor(label, dtype = torch.long).unsqueeze(0)
    input = torch.unsqueeze(input, 0)

    optimizer.zero_grad()      # 將梯度歸零

    y_pred = model(input)

    loss = criterion(y_pred, label)

    loss.backward()         # 透過 backward 取得每個參數的梯度值
    optimizer.step()         # 透過梯度下降執行下一步的參數更新

    train_loss_index += loss.item()*input.size(0)

  model.eval()

  def accuracy(x, y):
    for input, label in zip(x, y)                           # 遍歷所有資料

      output = model(input)

      _, pred = torch.max(output, 1)

      correct_tensor = pred.eq(label.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy())

      for i in range(batch_size):
          labels = label.data[i]
          class_correct[labels] += correct[i].item()
          class_total[labels] += 1                                  # 即預測正確correct累加


    for i in range(5):
      if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
      else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

    return np.sum(class_correct) / np.sum(class_total)                            # 計算準確率


  train_accuracy.append(accuracy(train_x, train_y))   # 續練資料帶入 ， (*100)是將結果轉成百分比的形式
  val_accuracy.append(accuracy(valid_x, valid_y))         # 驗證資料帶入 ， (*100)是將結果轉成百分比的形式


  for (input, label) in zip(valid_x, valid_y):
    label = torch.tensor(label, dtype = torch.long).unsqueeze(0)
    input = torch.unsqueeze(input, 0)

    y_pred = model(input)

    loss = criterion(y_pred, label)

    valid_loss_index += loss.item()*input.size(0)

  train_loss_index = train_loss_index/len(train_x)
  valid_loss_index = valid_loss_index/len(valid_x)

  train_loss.append(train_loss_index)
  valid_loss.append(valid_loss_index)

  if not (epoch + 1) % 1:
    print(f'Epoch: {epoch+1} \tTraining Loss: {train_loss_index} \tValidation Loss: {valid_loss_index}')
    #print(f'epoch: {epoch+1}, loss: {loss.item()}')

  early_stopping(valid_loss_index)
  if early_stopping.early_stop:
      print("Early stopping triggered.")
      break
print(model)




In [ ]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for input, label in test_loader:

    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(input)

    _, pred = torch.max(output, 1)

    correct_tensor = pred.eq(label.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())

    for i in range(batch_size):
        labels = label.data[i]
        class_correct[labels] += correct[i].item()
        class_total[labels] += 1

# average test loss
test_loss = test_loss/len(test_x)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data

def accuracy(images, labels):
    correct = 0
    for i in range(len(images)):                            # 遍歷所有資料

        images[i].reshape(784, 1)                           # 將images轉為列向量

        w = np.dot(w1, images[i]) + b1                      # w = Wx + b
       ly = Softmax(w)                                     # 計算Softmax

        a = np.argmax(ly)                                   # 取ly([x], [x], [x])中最大值的位置(索引)

        true = np.argmax(labels[i])                         # labels[x, x, x]中最大值的位置(索引)

        if (a == true):                                     # 如果兩者最大值位置相同
            correct += 1                                    # 即預測正確correct累加

    return correct / len(images)                            # 計算準確率

train_accuracy = accuracy(train_image, train_label) * 100   # 續練資料帶入 ， (*100)是將結果轉成百分比的形式
val_accuracy = accuracy(val_image, val_label) * 100         # 驗證資料帶入 ， (*100)是將結果轉成百分比的形式




for input, label in test_loader:

    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(input)

    _, pred = torch.max(output, 1)

    correct_tensor = pred.eq(label.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())

    for i in range(batch_size):
        labels = label.data[i]
        class_correct[labels] += correct[i].item()
        class_total[labels] += 1

# average test loss
test_loss = test_loss/len(test_x)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
import json
with open('123.json') as f:
    data = json.load(f)

print(label)

0    1
Name: Label, dtype: int64


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import json
import os

data = os.path.join("/content/drive/MyDrive/cifar-10/train.json")
df = pd.read_json(data)
label = df["Label"]
image = df["Image"].tolist()

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

y = label
x = []

for data in image:
  datas = np.array(data, dtype = np.uint8)
  datas = Image.fromarray(datas)
  x.append(transform(datas))


print(x)
print(type(x))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
import json
import os

data = os.path.join("/content/123.json")
df = pd.read_json(data)
label = df["Label"]
image = df["Image"].tolist()

transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.5, 0.5, 0.5],std = [0.5, 0.5, 0.5])
])
imgs = []

for a in image:
  aa = np.array(a, dtype = np.uint8)
  aa = Image.fromarray(aa)
  img = transform(aa)
  imgs.append(img)

  out_image = to_pil_image(img)
  out_path = os.path.join("/content/123.png")
  out_image.save(out_path)
  print(f"Saved image: {out_path}")

#print(f"Processed {len(imgs)} images and saved them.")
print(imgs)
print(type(imgs))